In [3]:
import flax.linen as fnn
import jax
import jax.numpy as jnp

In [16]:
class Critic(fnn.Module):

    def setup(self):
        self.n1 = fnn.Dense(10)
        self.n2 = fnn.Dense(10)

    def __call__(self,x,flag):
        """
        calculate q value

        Args:
            observations (AgentObservation): NamedTuple for observation of agent. consisting of basic observations and communication

        Returns:
            Array: q value
        """
        def _apply1(_,x):
            return self.n1(x)
        def _apply2(_,x):
            return self.n2(x)
        def _apply(x,flag,mdl):
            return fnn.cond(jnp.equal(flag,1), _apply1, _apply2,mdl, x)
        x = jax.vmap(_apply, in_axes=(0,0,None))(x,flag,self)

        return x

In [17]:
x = jnp.ones((2,5))
flag = jnp.array([0.0,1])
model = Critic()

In [18]:
params = model.init(jax.random.PRNGKey(0),x,flag)["params"]

JaxTransformError: Jax transforms and Flax models cannot be mixed. (https://flax.readthedocs.io/en/latest/flax.errors.html#flax.errors.JaxTransformError)

In [9]:
model.apply({"params":params}, x,flag)

UnexpectedTracerError: Encountered an unexpected tracer. A function transformed by JAX had a side effect, allowing for a reference to an intermediate value with shape (5, 10) and dtype float32 to escape.
JAX transformations require that functions explicitly return their outputs, and disallow saving intermediate values to global state.
To catch the leak earlier, try setting the environment variable JAX_CHECK_TRACER_LEAKS or using the `jax.checking_leaks` context manager.Detail: Different traces at same level: Traced<ShapedArray(float32[5,10])>with<BatchTrace(level=1/0)> with
  val = DeviceArray([[[-0.10984159, -0.8885538 , -0.38159436, -0.69824165,
                0.02169505, -0.6173719 , -0.480812  ,  0.31227043,
                0.5641242 , -0.29054263],
              [ 0.37325472,  0.25778595, -0.46811652, -0.6306121 ,
               -0.03621585, -0.19614427, -0.68257594, -0.4313423 ,
                0.66488945,  0.0259968 ],
              [ 0.70098555, -0.24303025, -0.5435464 ,  0.45